In [4]:
# ! pip install pyarrow fastparquet

In [88]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

**Answer**: 19

In [34]:
def read_dataset(fn):
    df = pd.read_parquet(fn)
    return df

In [35]:
df_january = read_dataset('yellow_tripdata_2022-01.parquet')
print('Number of columns in the January dataset: ', df_january.shape[1])

Number of columns in the January dataset:  19


In [36]:
df_january.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0


In [37]:
df_january.shape

(2463931, 19)

In [38]:
df_january.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

### Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

**Answer**: 46.45

In [39]:
# compute the duration variable
df_january['duration'] = df_january['tpep_dropoff_datetime'] - df_january['tpep_pickup_datetime']
df_january['duration']

0         0 days 00:17:49
1         0 days 00:08:24
2         0 days 00:08:58
3         0 days 00:10:02
4         0 days 00:37:32
                ...      
2463926   0 days 00:05:58
2463927   0 days 00:10:39
2463928   0 days 00:11:00
2463929   0 days 00:12:03
2463930   0 days 00:27:00
Name: duration, Length: 2463931, dtype: timedelta64[ns]

In [40]:
# convert timedelta64[ns] to seconds and then to minutes
df_january['duration'] = df_january['duration'].dt.total_seconds()/60
df_january['duration']

0          17.816667
1           8.400000
2           8.966667
3          10.033333
4          37.533333
             ...    
2463926     5.966667
2463927    10.650000
2463928    11.000000
2463929    12.050000
2463930    27.000000
Name: duration, Length: 2463931, dtype: float64

In [41]:
# calculate the standard deviation
round(np.std(df_january['duration']), 2)

46.45

### Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

**Answer**: 98%

In [58]:
# a new dataset without outliers
df_january_no_outliers = df_january[(df_january['duration']>=1) & (df_january['duration']<=60)]
df_january_no_outliers.shape

(2421440, 20)

In [59]:
df_january.shape

(2463931, 20)

In [63]:
print('The fraction of the records left: ', round(df_january_no_outliers.shape[0]/df_january.shape[0]*100, 1))

The fraction of the records left:  98.3


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

**Answer**: 515

In [71]:
location_ids = ['PULocationID', 'DOLocationID']
df_january_no_outliers[location_ids].head(3)

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166


In [76]:
df_january_no_outliers[location_ids].dtypes

PULocationID    int64
DOLocationID    int64
dtype: object

In [79]:
df_january_no_outliers[location_ids] = df_january_no_outliers[location_ids].astype(str)

/var/folders/dh/79f0200s5w5954gv_2lcc5g80000gn/T/ipykernel_56154/763846212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_january_no_outliers[location_ids] = df_january_no_outliers[location_ids].astype(str)


In [82]:
df_january_no_outliers[location_ids].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [84]:
january_dicts = df_january_no_outliers[location_ids].to_dict(orient='records')

In [85]:
len(january_dicts)

2421440

In [118]:
dv = DictVectorizer()

X_train = dv.fit_transform(january_dicts)

In [87]:
print('The dimensionality of the feature matrix: ', X_train.shape[1])

The dimensionality of the feature matrix:  515


### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

**Answer**: 6.99

In [89]:
lr = LinearRegression()
y_train= df_january_no_outliers['duration']

In [91]:
lr.fit(X_train, y_train)
train_pred = lr.predict(X_train)
rmse = round(mean_squared_error(y_train, train_pred, squared=False), 2)
print('RMSE on the train data: ', rmse)

RMSE on the train data:  6.99


### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

**Answer**: 7.79

In [122]:
df_february = read_dataset('yellow_tripdata_2022-02.parquet')
df_february.shape

(2979431, 19)

In [123]:
df_february['duration'] = df_february['tpep_dropoff_datetime'] - df_february['tpep_pickup_datetime']
df_february['duration'] = df_february['duration'].dt.total_seconds()/60

In [124]:
df_february = df_february[(df_february['duration']>=1) & (df_february['duration']<=60)]

In [125]:
df_february[location_ids] = df_february[location_ids].astype(str)

In [126]:
df_february[location_ids].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [127]:
february_dicts = df_february[location_ids].to_dict(orient='records')

In [128]:
len(february_dicts)

2918187

In [129]:
X_test = dv.transform(february_dicts)
X_test.shape

(2918187, 515)

In [130]:
y_test = df_february['duration']

In [131]:
test_pred = lr.predict(X_test)
rmse_test = round(mean_squared_error(y_test, test_pred, squared=False), 2)
print('RMSE on the test data: ', rmse_test)

RMSE on the test data:  7.79
